In [5]:
import pickle
import sys
import numpy as np
import matplotlib.pyplot as plt 
from pathlib import Path
import bgp_qnm_fits as bgp

In [6]:

SMOOTHNESS = 1e-3
TIME_SHIFT = 0 

# These determine the parameter and training range but do not have to match `analysis times' used later.

RESIDUAL_BIG_START = -10
RESIDUAL_BIG_END = 310
TIME_STEP = 0.1

TRAINING_START_TIME_WN = 0
TRAINING_RANGE_WN = 200

HYPERPARAM_RULE_DICT_WN = {
    "sigma_max": "multiply",
}

HYPERPARAM_RULE_DICT_GP = {
    "sigma_max": "multiply",
    "period": "multiply",
}

HYPERPARAM_RULE_DICT_GPC = {
    "sigma_max": "multiply",
    "period": "multiply",
    "length_scale_2": "multiply",
    "period_2": "multiply",
    "a": "replace",
}

TRAINING_SPH_MODES = [
    (2, 2),
    (2, 1),
    (3, 3),
    (3, 2),
    (4, 4),
    (2, -2),
    (2, -1),
    (3, -3),
    (3, -2),
    (4, -4),
]

SIM_TRAINING_MODE_RULES = {
    "0001": "PE",
    "0002": "PE",
    "0003": "PE",
    "0004": "PE",
    "0005": "P",
    "0006": "P",
    "0007": "P",
    "0008": "ALL",
    "0009": "E",
    "0010": "P",
    "0011": "P",
    "0012": "P",
    "0013": "ALL",
}

In [8]:
analysis_times = np.arange(TRAINING_START_TIME_WN, TRAINING_START_TIME_WN + TRAINING_RANGE_WN, TIME_STEP)

param_dict_sim_lm = bgp.get_param_dict(data_type="news")
tuned_params_news = bgp.get_tuned_param_dict("GP", data_type="news")
R_news = bgp.get_residual_data(big=True, data_type="news")

In [24]:
full_times = np.arange(RESIDUAL_BIG_START, RESIDUAL_BIG_START+RESIDUAL_BIG_END, TIME_STEP)
# Find the indices in full_times that are closest to the first and last analysis_times
start_idx = np.abs(full_times - analysis_times[0]).argmin()
end_idx = np.abs(full_times - analysis_times[-1]).argmin()
mask = np.zeros_like(full_times, dtype=bool)
mask[start_idx:end_idx+1] = True

R_news_masked = {
    key: {subkey: np.array(values)[mask] for subkey, values in subdict.items()}
    for key, subdict in R_news.items()
}

In [25]:
len(R_news_masked['0001'][(2,2)])

2000

In [26]:
wn_hyperparams = [0.20202315621798156]
gp_hyperparams = [6.916479343926145, 1.6788343847045542]
gpc_hyperparams = [8.28904248745442, 1.959288624090839, 0.4204419582784812, 1.0019473516595827, 0.11533919911875894]

In [27]:
likelihood_wn = bgp.get_total_log_likelihood(
    wn_hyperparams,
    param_dict_sim_lm,
    R_news_masked,
    HYPERPARAM_RULE_DICT_WN,
    analysis_times,
    bgp.kernel_WN,
    TRAINING_SPH_MODES,
    SIM_TRAINING_MODE_RULES,
    alpha=2,
    beta=2,
)

likelihood_gp = bgp.get_total_log_likelihood(
    gp_hyperparams,
    param_dict_sim_lm,
    R_news_masked,
    HYPERPARAM_RULE_DICT_GP,
    analysis_times,
    bgp.kernel_GP,
    TRAINING_SPH_MODES,
    SIM_TRAINING_MODE_RULES,
    alpha=2,
    beta=2,
)

likelihood_gp = bgp.get_total_log_likelihood(
    gp_hyperparams,
    param_dict_sim_lm,
    R_news_masked,
    HYPERPARAM_RULE_DICT_GP,
    analysis_times,
    bgp.kernel_GP,
    TRAINING_SPH_MODES,
    SIM_TRAINING_MODE_RULES,
    alpha=2,
    beta=2,
)

likelihood_gpc = bgp.get_total_log_likelihood(
    gpc_hyperparams,
    param_dict_sim_lm,
    R_news_masked,
    HYPERPARAM_RULE_DICT_GPC,
    analysis_times,
    bgp.kernel_GPC,
    TRAINING_SPH_MODES,
    SIM_TRAINING_MODE_RULES,
    alpha=2,
    beta=2,
)


In [28]:
print(likelihood_wn)
print(likelihood_gp)
print(likelihood_gpc)

-4312853.507679867
-7338376.2073224615
-7347277.18159403
